# QM9 KRR Benchmark

Code to perform KRR on the QM9 dataset using different Nystrom methods, using 100k randomly selected molecules as training points. The l1 Laplace kernel is used with a bandwidth 5120, and the regularization parameter is 1e-8; both were chosen using cross-validation. This code was used to produce Figure 3 in the manuscript together with `matlab_plotting/make_krr_plots.m`

In [1]:
# %load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [13]:
import sys
sys.path.append('../')

import qml, os
from scipy.io import savemat, loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.express as px

from KRR_Nystrom import KRR_Nystrom
import rpcholesky
import leverage_score
import unif_sample
import matplotlib.pyplot as plt
import time
from functools import partial
import pickle

In [28]:
# add this to be able to render plotly plots in non-vscode notebooks
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:

def get_molecules(directory = "molecules/", max_atoms = 29, max_mols = np.Inf, output_index = 7):
    compounds = []
    energies = []
    for f in sorted(os.listdir("molecules/")):
        if len(compounds) >= max_mols:
            break

        try:
            mol = qml.Compound(xyz="molecules/"+f)
            mol.generate_coulomb_matrix(size=max_atoms, sorting="row-norm")
            with open("molecules/"+f) as myfile:
                line = list(myfile.readlines())[1]
                energies.append(float(line.split()[output_index]) * 27.2114) # Hartrees to eV
            compounds.append(mol)
        except ValueError:
            pass
    
    c = list(zip(compounds, energies))
    np.random.shuffle(c)
    compounds, energies = zip(*c)

    X = np.array([mol.representation for mol in compounds])
    Y = np.array(energies).reshape((X.shape[0],1))

    return X, Y 
    

In [4]:
if not os.path.isfile("data/homo.mat"):
    X, Y = get_molecules()
    data = { "X" : X, "Y" : Y }
    savemat("data/homo.mat", data)
else:
    data = loadmat("data/homo.mat")
    
feature = data['X']
target = data['Y'].flatten()
scaler = StandardScaler()
feature = scaler.fit_transform(feature)
n,d = np.shape(feature)


In [5]:
# From KT compress
def log4(n):
    return np.log2(n) / 2
def get_g(n):
    return int( np.ceil( log4(log4(n)) ) ) # Use default value
def largest_power_of_four(n):
    """Returns largest power of four less than or equal to n
    """
    return 4**( (n.bit_length() - 1 )//2)

def get_coreset_size(n, m=1):
    if get_g(n) <= m:
        # with TicToc('compresspp', print_toc=PRINT_TOC):
        # Compress with g'=g+inflation (compressing returns set of size 2^(g+inflation) sqrt(n) )
        # Thin with g'=g (thinning returns set of size 2^inflation sqrt(n) )
        largest_pow_four = largest_power_of_four(n)
        log2n = n.bit_length() - 1
        scale = n // largest_pow_four
        return 2**( 2*(log2n//2) - m ) * scale
    else:
        return int(n / 2**m)

In [6]:

num_train = 20000 # 100000
num_test = 20000 # n - num_train
# ks = range(200, 1200, 200)
sqrt_n = get_coreset_size(num_train, int(log4(num_train)))
ks = [ sqrt_n, sqrt_n* 2, sqrt_n*4 ]
print('ks', ks)

train_sample = feature[:num_train]
train_sample_target = target[:num_train]
test_sample = feature[num_train:num_train+num_test]
test_sample_target = target[num_train:num_train+num_test]


ks [128, 256, 512]


In [7]:

def mean_squared_error(true, pred):
    return np.mean((true - pred)**2)
def mean_average_error(true, pred):
    return np.mean(np.abs(true - pred))
def SMAPE(true,pred):
    return np.mean(abs(true - pred)/((abs(true)+abs(pred))/2))


In [8]:

methods = { 
    # deterministic methods
    'Greedy' : rpcholesky.greedy, 
    
    # random methods
    'Uniform' : unif_sample.uniform_sample,
    'RPCholesky' : rpcholesky.rpcholesky,
    'RLS' : leverage_score.recursive_rls_acc,
    'block50RPCholesky' : partial(rpcholesky.block_rpcholesky,b=50)
}

num_trials = 100
lamb = 1.0e-8
sigma = 5120.0
result = dict()

solve_method = 'Direct'

for name, method in methods.items():
    result[name] = dict()
    print(f'------------- Method: {name} -------------')
    result[name]["trace_errors"] = {} #np.zeros((len(ks),2))
    result[name]["KRRMSE"] = {} #np.zeros((len(ks),2))
    result[name]["KRRMAE"] = {} #np.zeros((len(ks),2))
    result[name]["KRRSMAPE"] = {}   #np.zeros((len(ks),2))
    result[name]["queries"] = {}    #np.zeros((len(ks),2))

    for idx_k in range(len(ks)):
        k = ks[idx_k]
        print(f'k = {k}')
        trace_err = []
        runtime = []
        queries = []
        KRRmse = []
        KRRmae = []
        KRRsmape = []
        if "Greedy" not in name:
            for i in range(num_trials):
                while True:
                    try:
                        print(f"Trial {i}")
                        # # Original
                        # model = KRR_Nystrom(kernel = "gaussian", bandwidth = sigma)
                        # # Bug fix
                        model = KRR_Nystrom(kernel = "laplace", bandwidth = sigma)
                        
                        model.fit_Nystrom(train_sample, train_sample_target, lamb = lamb, sample_num = k, sample_method = method, solve_method = solve_method)
                        preds = model.predict_Nystrom(test_sample)
                        break
                    except np.linalg.LinAlgError:
                        continue
                KRRmse.append(mean_squared_error(test_sample_target, preds))
                KRRmae.append(mean_average_error(test_sample_target, preds))
                KRRsmape.append(SMAPE(test_sample_target, preds))
                queries.append(model.queries)
                trace_err.append(model.reltrace_err)  

                print(f'KRR acc: mse {KRRmse[-1]}, mae {KRRmae[-1]}, smape {KRRsmape[-1]}')
                print(f'time: sample {model.sample_time} s, linsolve {model.linsolve_time} s, pred {model.pred_time} s')

        else:
            model = KRR_Nystrom(kernel = "laplace", 
                        bandwidth = sigma)
            model.fit_Nystrom(train_sample, train_sample_target, lamb = lamb, sample_num = k, sample_method = method, solve_method = solve_method)
            preds = model.predict_Nystrom(test_sample)
            KRRmse.append(mean_squared_error(test_sample_target, preds))
            KRRmae.append(mean_average_error(test_sample_target, preds))
            KRRsmape.append(SMAPE(test_sample_target, preds))
            queries.append(model.queries)
            trace_err.append(model.reltrace_err) 

            print(f'KRR acc: mse {KRRmse[-1]}, mae {KRRmae[-1]}, smape {KRRsmape[-1]}')
            print(f'time: sample {model.sample_time}, linsolve {model.linsolve_time}, pred {model.pred_time}')

        result[name]["trace_errors"][k] = trace_err   # [np.mean(trace_err),np.std(trace_err)]
        result[name]["KRRMSE"][k] = KRRmse    # [np.mean(KRRmse),np.std(KRRmse)]
        result[name]["KRRMAE"][k] = KRRmae    #[np.mean(KRRmae),np.std(KRRmae)]
        result[name]["KRRSMAPE"][k] = KRRsmape    #[np.mean(KRRsmape),np.std(KRRsmape)]
        result[name]["queries"][k] = queries  #[np.mean(queries)/float(num_train**2),np.std(queries)/float(num_train**2)]

        # savemat("data/{}_molecule100k.mat".format(name), result[name])
        # use pickle to save the result periodically
        with open(f"data/molecule{num_train // 1000}k.pkl", 'wb') as f:
            pickle.dump(result, f)


------------- Method: Greedy -------------
k = 128


KRR acc: mse 0.32869427987617694, mae 0.4207989561384129, smape 0.06522165673026105
time: sample 4.638573408126831, linsolve 0.015665054321289062, pred 0.39659953117370605
k = 256


KRR acc: mse 0.2729237221965333, mae 0.3884841928324148, smape 0.060296657775134974
time: sample 9.168869733810425, linsolve 0.01515054702758789, pred 0.7583320140838623
k = 512


KRR acc: mse 0.23277200066947504, mae 0.3586732396610669, smape 0.05570710395147019
time: sample 18.69033718109131, linsolve 0.042438507080078125, pred 1.5357978343963623
------------- Method: Uniform -------------
k = 128
Trial 0


KRR acc: mse 0.26385920203274843, mae 0.37818448220893397, smape 0.0587607301953572
time: sample 0.5500543117523193 s, linsolve 0.0029168128967285156 s, pred 0.3948383331298828 s
Trial 1


...

KRR acc: mse 0.1864021680492731, mae 0.3244572813433044, smape 0.05034756352335777
time: sample 2.6041646003723145 s, linsolve 0.029629945755004883 s, pred 1.536219596862793 s
Trial 96


KRR acc: mse 0.19115565110446522, mae 0.32844599679262226, smape 0.050942274820257566
time: sample 2.5922176837921143 s, linsolve 0.029561758041381836 s, pred 1.5351033210754395 s
Trial 97


KRR acc: mse 0.18710221470268618, mae 0.3264417561853452, smape 0.05063968945448219
time: sample 2.6188364028930664 s, linsolve 0.029858112335205078 s, pred 1.5340826511383057 s
Trial 98


KRR acc: mse 0.18415727804091003, mae 0.3236111088164506, smape 0.05023512902281209
time: sample 2.7853472232818604 s, linsolve 0.029635906219482422 s, pred 1.53379225730896 s
Trial 99


KRR acc: mse 0.18870326713236557, mae 0.32657661849293684, smape 0.05071236743123013
time: sample 2.622714042663574 s, linsolve 0.029421091079711914 s, pred 1.535161018371582 s


In [9]:
result

{'Greedy': {'trace_errors': {128: [0.044340903098873785],
   256: [0.03172596166304484],
   512: [0.020065985813392945]},
  'KRRMSE': {128: [0.32869427987617694],
   256: [0.2729237221965333],
   512: [0.23277200066947504]},
  'KRRMAE': {128: [0.4207989561384129],
   256: [0.3884841928324148],
   512: [0.3586732396610669]},
  'KRRSMAPE': {128: [0.06522165673026105],
   256: [0.060296657775134974],
   512: [0.05570710395147019]},
  'queries': {128: [2580000], 256: [5140000], 512: [10260000]}},
 'Uniform': {'trace_errors': {128: [0.022947270399702575,
    0.023595387809545537,
    0.023255985258751933,
    0.022897990427619332,
    0.02370821400363875,
    0.023579659607831125,
    0.02349022437548883,
    0.023217748899482468,
    0.022788799903144717,
    0.022732987444361606,
    0.023320958097323274,
    0.022993991203985025,
    0.023458599036543636,
    0.023139437809184527,
    0.023335842018536824,
    0.02351449836281954,
    0.02364153327468448,
    0.02331660494668522,
    0.0

In [7]:
with open("data/molecule20k.pkl", "rb") as f:
    result = pickle.load(f)

## Plot results

In [29]:
# plot line graph with error bars
metrics = ['KRRSMAPE', 'KRRMSE']
fig = make_subplots(rows=len(metrics), cols=1, subplot_titles=metrics, shared_xaxes=True, vertical_spacing=0.1)

for name in result.keys():
    for r, metric in enumerate(metrics):

        means = [np.mean(krrmse) for _, krrmse in result[name][metric].items() ]
        stds = [np.std(krrmse) for _, krrmse in result[name][metric].items() ]
        # print(ks, means, stds)
        fig.add_trace(go.Scatter(
            x=ks, 
            y=means, 
            mode='lines+markers', 
            name=name, 
            error_y=dict(
                type='data', # value of error bar given in data coordinates
                array=stds,
                visible=True
            ),
            legendgroup=name,
        ), row=r+1, col=1)
    fig.update_xaxes(title_text="k", row=r+1, col=1)

        
fig.update_layout(title='MSE vs. k (columns)',
                    #  xaxis_title='Coreset Size',
                    #  yaxis_title='MSE',
                     height=800, width=800)
fig.show()